In [9]:
import importlib

import src.valid_index
importlib.reload(src.valid_index)
from src.valid_index import get_excon_manual_index

import src.file_tools
importlib.reload(src.file_tools)
from src.file_tools import read_processed_regs_into_dataframe

import src.embeddings
importlib.reload(src.embeddings)
from src.embeddings import get_ada_embedding

Get the files that make up the raw regulations text

In [2]:
import os
import fnmatch

dir_path = './manual_test/' # Dummy Data
#dir_path = './manual/' # Actual Files
file_list = []
for root, dir, files in os.walk(dir_path):
    for file in files:
        str = 'excon_manual*.txt'
        if fnmatch.fnmatch(file, str):
            file_path = os.path.join(root, file)
            file_list.append(file_path)


In [3]:
excon_index = get_excon_manual_index()

non_text_labels = ['Table', 'Formula', 'Example', 'Definition']
print_summary_stats = True
df_excon, non_text = read_processed_regs_into_dataframe(file_list=file_list, valid_index_checker=excon_index, non_text_labels=non_text_labels, print_summary=print_summary_stats)

total lines in dataframe:  59
total  Table :  1
total  Formula :  0
total  Example :  0
total  Definition :  2


In [4]:
df_excon.to_csv("./manual_test/excon_processed_manual_test.csv", sep='|', encoding="utf-8", index=False)

Create parquet file with some questions and their mappings

In [15]:
import pandas as pd
list_of_questions = []
list_of_questions.append(["C.(A)", "Export of gold jewellery by manufacturing jewellers", "heading"])
list_of_questions.append(["C.(B)", "Export of gold other than jewellery", "heading"])
list_of_questions.append(["C.(C)", "Acquisition of gold for trade purposes", "heading"])
list_of_questions.append(["C.(D)", "Advances against gold", "heading"])
list_of_questions.append(["C.(E)", "Krugerrand coins", "heading"])
list_of_questions.append(["C.(F)", "Marketing of South African gold", "heading"])
list_of_questions.append(["C.(G)", "Applications for the importation of gold", "heading"])
df_text_test = pd.DataFrame(list_of_questions, columns = ["section", "text", "source"])

df_text_test["Embedding"] = df_text_test['text'].apply(get_ada_embedding)


In [18]:
list_of_definitions = []
list_of_definitions.append(["all", "Authorised Dealer means, in relation to any transaction in respect of gold, a person authorised by the Financial Surveillance Department to deal in gold and, in relation to any transaction in respect of foreign exchange, a person authorised by the Financial Surveillance Department to deal in foreign exchange."])
list_of_definitions.append(["all", "Gold as referred to in Regulations 2 and 5 includes all forms of gold other than wrought gold, as well as ingots, amalgam, concentrates or salts of gold buttons and trade scrap. Gold as referred to in Regulation 3 includes wrought gold and gold coins."])
df_definitions_test = pd.DataFrame(list_of_definitions, columns = ["Definition", "source"])

df_definitions_test["Embedding"] = df_definitions_test["Definition"].apply(get_ada_embedding)
df_definitions_test.to_parquet("./inputs_test/definitions.parquet", engine='pyarrow')


In [14]:
df_definitions = pd.read_parquet("./inputs/definitions.parquet", engine='pyarrow')
df_definitions


,Definition,Embedding,source
0,"Treasury means, in relation to any matter cont...","[-0.010001085698604584, -0.04355744272470474, ...",all
1,ADLA Manual means the Currency and Exchanges M...,"[0.0025530867278575897, -0.007961710914969444,...",all
2,"Affected person means a body corporate, founda...","[-0.026521744206547737, -0.020068032667040825,...",all
3,Africa means any country forming part of the A...,"[-0.00920949224382639, -0.01762852445244789, 0...",all
4,"Authorised Dealer means, in relation to any tr...","[-0.007704835385084152, -0.005596109200268984,...",all
...,...,...,...
85,(jj) Settlement authority means the person or ...,"[-0.01835245080292225, 0.005121075082570314, -...",securities
86,(kk) Stockbroker means a natural person who is...,"[-0.02447284199297428, -0.011708551086485386, ...",securities
87,(ll) Stock exchange means any association lice...,"[-0.024108372628688812, -0.015252502635121346,...",securities
88,(mm) STRATE means Share Transactions Totally E...,"[-0.025344470515847206, -0.010574902407824993,...",securities


In [7]:
import pandas as pd
text_and_embeddings_file = "./inputs/text.parquet"
df_text_all = pd.read_parquet(text_and_embeddings_file, engine='pyarrow')

In [8]:
df_text_all

,section,text,Embedding,source
0,Legal context,Legal context,"[0.008710513822734356, -0.01103331707417965, 0...",heading
1,Introduction,Introduction,"[0.007698623929172754, -0.004689161665737629, ...",heading
2,A.1,Definitions,"[-0.005661599338054657, -0.014509140513837337,...",heading
3,A.2(A),Authorised entities. Authorised Dealers,"[-0.009998462162911892, -0.009398686699569225,...",heading
4,A.2(B),Authorised entities. Authorised Dealers in for...,"[-0.009065092541277409, -0.012516901828348637,...",heading
...,...,...,...,...
1497,I.3(C)(ii),What type of assets can be financed by the sho...,"[-0.010561777278780937, -0.010366550646722317,...",question
1498,I.3(C)(iii),Can an authorised dealer assume local funding ...,"[0.014643068425357342, -0.006568756885826588, ...",question
1499,I.3(C)(iii),What is required for an authorised dealer to u...,"[0.015578720718622208, -0.01724221184849739, 0...",question
1500,I.3(D),What type of debt can Authorised Dealers repay?,"[-0.0073911575600504875, 0.005508958827704191,...",question
